In [ ]:
#python3 t2_split.py --cfv 2 --fromf ts2_16x260_inv

In [1]:
import os, gc, subprocess, time, sys, shutil, argparse

import scipy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
#from tqdm.notebook import tqdm
import skimage.io
from skimage.transform import resize, rescale
from math import sqrt
import statistics

sys.path.append("..")

In [2]:
import PIL.Image
#PIL.Image.MAX_IMAGE_PIXELS = 400000000
PIL.Image.MAX_IMAGE_PIXELS = None

In [4]:
from panda_bvv_config import *

## 1. Inputs ##

In [5]:
Zoom = 2

#these 2 values are for x5 cross-fold validation:
choices_=[0, 1, 2, 3, 4, 5]
cfv_const= -2123
last_train_label = 7500
last_valid_label = 9400


In [ ]:
ap = argparse.ArgumentParser()
ap.add_argument('--cfv', help="cross fold validation number", dest = 'cfv', default = 1,\
                type=int, choices=choices_)
ap.add_argument('--fromf', help="from folder name", dest = 'fromf',\
                type=str)
                #("--cnn", type=str, help="training cnn name", dest = 'train_cnn')
args = vars(ap.parse_args())

In [ ]:
#data-from folder:
new_proc_folder = args["fromf"]
cfv = args["cfv"]

In [7]:
# wrapping inputs:
new_proc_folder = os.path.join(base_path, new_proc_folder)


## 2. Standard functions ##

## 5. Distribute processed files among training and validation folders ##

In [21]:
try:
    shutil.rmtree(train_cnn)
    shutil.rmtree(valid_cnn)
except OSError as ee:
    print ("Error: %s: " % ee)
os.mkdir(train_cnn)
os.mkdir(valid_cnn)
for grade in isup_classes:
    os.mkdir(os.path.join(train_cnn, f'isup{grade}'))
    os.mkdir(os.path.join(valid_cnn, f'isup{grade}'))
# 

In [29]:
#cross-fold validation:
labels_perm = train_labels.sample(frac = 1, random_state=npseed)
if cfv == 0: #take data without test:    
    train_cnn_labels = labels_perm[:last_train_label]
    valid_cnn_labels = labels_perm[last_train_label:last_valid_label]
elif cfv == 1:
    train_cnn_labels = labels_perm[:cfv_const]
    valid_cnn_labels = labels_perm[cfv_const:]
else:
    train_cnn_labels = pd.concat([labels_perm[:cfv_const*cfv], labels_perm[cfv_const*(cfv-1):]], sort=False)
    valid_cnn_labels = labels_perm[cfv_const*cfv:cfv_const*(cfv-1)]

In [23]:
#split train files among ISUP categories
with tqdm(total=train_cnn_labels.shape[0]) as pbar:
    for j, row in train_cnn_labels.iterrows():
        filename = row['image_id'] + '.png'       
        path_to_train_class = os.path.join(train_cnn, 'isup' + str(row['isup_grade']), filename)
        path_to_copy_from = os.path.join(new_proc_folder, filename)
        bashCommand = "cp " + path_to_copy_from + ' ' + path_to_train_class
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        if error: print(str(error))
        pbar.update(1)


CPU times: user 8.7 s, sys: 15.9 s, total: 24.6 s
Wall time: 1min 54s


In [24]:
#split validation files among ISUP categories
with tqdm(total=valid_cnn_labels.shape[0]) as pbar:
    for j, row in valid_cnn_labels.iterrows():
        filename = row['image_id'] + '.png'
        path_to_valid_class = os.path.join(valid_cnn, 'isup' + str(row['isup_grade']), filename)    
        path_to_copy_from = os.path.join(new_proc_folder, filename)
        bashCommand = "cp " + path_to_copy_from + ' ' + path_to_valid_class    
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        if error: print(str(error))
        pbar.update(1)


CPU times: user 2.16 s, sys: 3.97 s, total: 6.12 s
Wall time: 28.8 s


In [1]:
import os
module_name = 't2_split'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)